# Buggy Coders - Whale A call recognition

In [ ]:
# the libararies imported are as follows
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
import aifc
from tqdm import tqdm
from pydub import AudioSegment
import numpy as np
import glob
from skimage.transform import resize
from sklearn.model_selection import GridSearchCV
import pylab as pl
from sklearn.metrics import f1_score
from matplotlib import mlab
import pandas as pd
import keras
from joblib import dump, load
from keras.optimizers import SGD, Adam, Adagrad, Adadelta, RMSprop, Nadam, Adamax, Ftrl
from keras.layers import Dense, Dropout, Activation, BatchNormalization, Flatten, Conv2D, MaxPooling2D
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import make_scorer, f1_score, log_loss
from keras.wrappers.scikit_learn import KerasClassifier
from keras.callbacks import Callback
from keras.models import load_model, Sequential
from keras.utils import plot_model
from matplotlib.pyplot import imshow
import keras.backend as K
K.set_image_data_format('channels_last')
import warnings
# Suppress warnings
warnings.filterwarnings('ignore')

In [ ]:
# convert all wav files to aiff files using audio segment package

AudioSegment.converter = "/usr/bin/ffmpeg"  # using ffmpeg package as a dependency for audio segment


def convert_to_aiff(file_path, new_file_path):
    # Load the .wav file
    audio = AudioSegment.from_wav(file_path)
    # Save the .aiff file
    audio.export(new_file_path, format="aiff")

# Converting training files
# List the directories containing the .wav files
labels = ["1", "0"]
to_directory = "aiff/train/"
directories = ["train/train/0", "train/train/1"]

# Get a list of all the .wav files in the directory
for directory in directories:
  label = directory[-1]
  wav_files = [f for f in os.listdir(directory) if f.endswith(".wav")]
  # Loop through each .wav file
  for wav_file in tqdm(wav_files, desc=directory):
      # Get the file path of the .wav file
      file_path = os.path.join(directory, wav_file)
      to_file_path = os.path.join(to_directory, wav_file)
      # Get the new file path for the .aiff file
      new_file_path = os.path.splitext(to_file_path)[0] + label + ".aiff"
      # Convert the .wav file to .aiff format
      convert_to_aiff(file_path, new_file_path)


# Converting test files
to_test_directory = "aiff/test/"
from_test_directory = "test/test"
# Get a list of all the .wav files in the from_test_directory
wav_files = [f for f in os.listdir(from_test_directory) if f.endswith(".wav")]
# Loop through each .wav file
for wav_file in tqdm(wav_files, desc=from_test_directory):
    # Get the file path of the .wav file
    file_path = os.path.join(from_test_directory, wav_file)
    to_file_path = os.path.join(to_test_directory, wav_file)
    # Get the new file path for the .aiff file
    new_file_path = os.path.splitext(to_file_path)[0] + ".aiff"
    # Convert the .wav file to .aiff format
    convert_to_aiff(file_path, new_file_path)

In [ ]:
"""
1. Pre Procesing:

We performed a pre-processing step to improve the horizontal (temporal dimension) and vertical (frequency dimension)
contrast of the spectrogram images which proved to be efficient in getting better results. Both kinds of contrast enhance-
ment were achieved by creating two moving average filters with only difference in their length (denoted filter A and filter B).

1.1 Temporal dimension:

Each row of the spectrogram was combined once with filter A and then with filter B. Since filter a had a much smaller length
compared to filter B the values of the output of the combination with filter A represented local averages for adjacent pixels
whereas the output of the combination with filter B represented global averages for neighborhoods of pixels. Contrast
enhancement was achieved by subtracting out these local averages from their corresponding rows, thereby emphasizing
more significant temporal changes.Contrast enhancement was performed by subtracting out the local averages from their
corresponding rows, thereby emphasizing more prominent temporal changes.

1.2 Frequency domain:

Each column of the spectrogram was combined sperately with filter A and filter B for contrast enhancement and to empha-
size more on significant differences in the frequency power distribution for every time step, the output of the combination
operation with filter A is subtracted from that of filter B, and the actual local averages subtracted from each corresponding
row.

Two new kinds of spectrogram images are produced from one original sample, thus doubling the number of samples in the
dataset.
"""
def ReadAIFF(file):
    ''' ReadAIFF Method
            Read AIFF and convert to numpy array
            
            Args: 
                file: string file to read 
            Returns:
                numpy array containing whale audio clip      
                
    '''
    s = aifc.open(file,'r')
    nFrames = s.getnframes()
    strSig = s.readframes(nFrames)
    return np.frombuffer(strSig,np.short).byteswap()

def SpecGram(file,params=None):
    '''  The  function takes an audio file (specified as a string file), and creates a spectrogram representation of it.
         It  pre-process input for input shape uniformity 
         Args: 1. string file to read 
               2. The parameters for creating the spectrogram are passed as a dictionary in the "params" argument.  
         The output of the function is a pre-processed spectrogram matrix and arrays for the frequency and time bins, which are one-dimensional.
                
    '''
    s = ReadAIFF(file)
    # Convert to spectrogram 
    P,freqs,bins = mlab.specgram(s,**params)
    m,n = P.shape
    # Ensure all image inputs to the CNN are the same size. If the number of time bins 
    # is less than 59, pad with zeros 
    if n < 59:
        Q = np.zeros((m,59))
        Q[:,:n] = P
    else:
        Q = P
    return Q,freqs,bins

def slidingWindowV(P,inner=3,outer=64,maxM=50,minM=7,maxT=59,norm=True):
    ''' Enhance the contrast along frequency dimension '''

    Q = P.copy()
    m, n = Q.shape
    if norm:
        #segment to remove  extreme values 
        mval, sval = np.mean(Q[minM:maxM,:maxT]), np.std(Q[minM:maxM,:maxT])
        fact_ = 1.5
        Q[Q > mval + fact_*sval] = mval + fact_*sval
        Q[Q < mval - fact_*sval] = mval - fact_*sval
        Q[:minM,:] = mval
    # Setting up the local mean window 
    wInner = np.ones(inner)
    # Setting up the overall mean window 
    wOuter = np.ones(outer)
    # Removing  overall mean and local mean using np.convolve 
   
    for i in range(maxT):
        Q[:,i] = Q[:,i] - (np.convolve(Q[:,i],wOuter,'same') - np.convolve(Q[:,i],wInner,'same'))/(outer - inner)
    Q[Q < 0] = 0.
    return Q[:maxM,:]

def slidingWindowH(P,inner=3,outer=32,maxM=50,minM=7,maxT=59,norm=True):
    ''' Enhance the contrast along temporal dimension '''
    Q = P.copy()
    m, n = Q.shape
    if outer > maxT:
        outer = maxT
    if norm:
        # Cutting off extreme values 
        mval, sval = np.mean(Q[minM:maxM,:maxT]), np.std(Q[minM:maxM,:maxT])
        fact_ = 1.5
        Q[Q > mval + fact_*sval] = mval + fact_*sval
        Q[Q < mval - fact_*sval] = mval - fact_*sval
        Q[:minM,:] = mval
    # Setting up the local mean window and overall mean window 
    wInner = np.ones(inner)
    wOuter = np.ones(outer)
    if inner > maxT:
        return Q[:maxM,:]
    # Removing overall mean and local mean using np.convolve     
    
    for i in range(maxM):
        Q[i,:maxT] = Q[i,:maxT] - (np.convolve(Q[i,:maxT],wOuter,'same') - np.convolve(Q[i,:maxT],wInner,'same'))/(outer - inner)
    Q[Q < 0] = 0.
    return Q[:maxM,:]


def get_file_id(file):
    id,extension = os.path.splitext(file)
    return id

def extract_labels(file):
    name,extension = os.path.splitext(file)
    label = name[-1]
    return int(label)

def extract_featuresV(file,params=None):
    '''The function is used for obtaining a spectrogram representation of an audio file with vertically-enhanced contrast.
       suitable for input into a Convolutional Neural Network (CNN).
       The audio file is specified as a string in the "file" argument,
       and the spectrogram parameters are passed in as a dictionary in the "params" argument. 
       The output of the function is a 2-dimensional numpy array, which is an image with vertically-enhanced contrast.    
                
    '''
    P,freqs,bins = SpecGram(file,params)
    Q = slidingWindowV(P,inner=3,maxM=50,maxT=bins.size)
    # Resize spectrogram image into a square matrix 
    Q = resize(Q,(64,64),mode='edge')
    return Q

def extract_featuresH(file,params=None):
    ''' The function is used  for obtaining a spectrogram of an audio file with an emphasis on horizontal contrast, 
       suitable for input into a Convolutional Neural Network (CNN). 
       The audio file is specified as a string in the "file" argument, and 
       the parameters for creating the spectrogram are passed in as a dictionary in the "params" argument. 
       The result of the function is a 2-dimensional numpy array, which is an image with horizontally-enhanced contrast. 
                 
                
    '''
    P,freqs,bins = SpecGram(file,params)
    W = slidingWindowH(P,inner=3,outer=32,maxM=50,maxT=bins.size)
    # Resize spectrogram image into a square matrix 
    W= resize(W,(64,64),mode='edge')
    return W


def get_training_data():
    ''' method of obtaining data'''

    # Spectrogram parameters 
    params = {'NFFT':256,'Fs':2000,'noverlap':192}
    # Load in the audio files from the training dataset
    path = 'aiff/train'
    filenames = glob.glob(path+'/*.aiff')
    """
     For each audio file, we obation  the spectrograms with vertically-enhanced contrast 
     and the spectrograms with horizontally-enhanced contrast. This in 
     effect doubles the amount of data for training, and presents the CNN with different 
     perspectives of the same spectrogram image of the original audio file 

    """

    print('Extracting Training Features')
    training_featuresV = np.array([extract_featuresV(x,params=params) for x in tqdm(filenames, desc="Extracting test features V")])
    training_featuresH = np.array([extract_featuresH(x,params=params) for x in tqdm(filenames, desc="Extracting test features H")])
    # Concatenate the two feature matrices together to form a double-length feature matrix
    X_train = np.append(training_featuresV,training_featuresH,axis=0)
    # Axis 0 indicates the number of examples, Axis 1 and 2 are the features (64x64 image
    # spectrograms). Add Axis 3 to indicate 1 channel (depth of 1 for spectrogram image) for 
    # compatibility with Keras CNN model 
    print(X_train.shape)
    X_train = X_train[:,:,:,np.newaxis]
    
    
    # Extract labels for the training dataset. Since the vertically-enhanced and 
    # horizontally-enhanced images are concatenated to form a training dataset twice as long,
    # append a duplicate copy of the training labels to form a training label vector 
    # twice as long 

    print('Extracting Training Labels')
    Y_train = np.array([extract_labels(x) for x in tqdm(filenames, desc="Extracting training labels")])
    Y_train = np.append(Y_train,Y_train)


    # Do not append a duplicate copy of the test labels to form a test label vector twice
    # as long, since the two feature matrices were not concatenated together previously. 
    # The number of elements in the test label vector is the number of original audio
    # files in the test dataset 

    return X_train, Y_train

def get_test_data():
  
    # Spectrogram parameters 
    params = {'NFFT':256,'Fs':2000,'noverlap':192}
    # Load in the audio files from the test dataset
    path = 'aiff/test'
    # filenames = glob.glob(path+'/[10]/*.aiff')
    filenames = glob.glob(path+'/*.aiff')

    # For each audio file, extract the spectrograms with vertically-enhanced contrast 
    # separately from the spectrograms with horizontally-enhanced contrast. This in 
    # effect doubles the amount of data for training, and presents the CNN with different 
    # perspectives of the same spectrogram image of the original audio file 


    print('Extracting File id')
    file_id = np.array([get_file_id(x) for x in tqdm(filenames, desc="Extracting File id")])
    file_id = np.append(file_id,file_id)


    print('Extracting Test Features')
    X_testV = np.array([extract_featuresV(x,params=params) for x in tqdm(filenames, desc="Extracting test features V")])
    X_testH = np.array([extract_featuresH(x,params=params) for x in tqdm(filenames, desc="Extracting test features H")])
    X_testV = X_testV[:,:,:,np.newaxis]
    X_testH = X_testH[:,:,:,np.newaxis]
    X_test = np.append(X_testV,X_testV,axis=0)
    X_test = X_test[:,:,:,np.newaxis]


    return file_id, X_test, X_testV, X_testH


X_train, Y_train = get_training_data()
file_id, X_test, X_testV, X_testH = get_test_data()


In [ ]:
"""
A spectrogram was obtained from every audio file in the training set The spectrogram’s contrast was enhanced hori-
zontally (temporal dimension) and vertically (frequency dimension) by removing extreme values and implementing
a sliding mean.
• The CNN model’s hyperparameters were optimized using 3-Fold Cross Validation via GridSearchCV. The CNN
was fit using the enhanced training set
• .If the vertically-enhanced input yielded 1 and the horizontally-enhanced input yielded 0, the final predicted label
was 1.
• We choosed The Receiver Operating Characteristic (ROC) for evaluation, being a measure of the true positive rate vs
false positive rate as the discrimination threshold of the binary classifier is varied. The area under the curve (AUC)
is a single number metric of a binary classifier’s ROC curve and it is this ROC-AUC score that is used for evaluation
of the CNN model.
"""
class CNNModel(object):
    def __init__(self, learning_rate=0.01, activation='relu', optimizer=SGD):
        self.learning_rate = learning_rate
        self.activation = activation
        self.optimizer = optimizer
        self.classes_ = [0, 1]

    def create_model(self):
        model = Sequential()
        # Dropout on the visible layer (1 in 5 probability of dropout) 
        model.add(Dropout(0.2,input_shape=(X_train.shape[1],X_train.shape[2],X_train.shape[3]),name='drop1'))
        # Conv2D -> BatchNorm -> Relu Activation -> MaxPooling2D
        model.add(Conv2D(15,(7,7),strides=(1,1),name='conv1'))
        model.add(BatchNormalization(axis=3,name='bn1'))
        model.add(Activation('relu'))
        model.add(MaxPooling2D((2,2),name='max_pool1'))
        # Conv2D -> BatchNorm -> Relu Activation -> MaxPooling2D
        model.add(Conv2D(30,(7,7),strides=(1,1),name='conv2'))
        model.add(BatchNormalization(axis=3,name='bn2'))
        model.add(Activation('relu'))
        model.add(MaxPooling2D((2,2),name='max_pool2'))
        # Flatten to yield input for the fully connected layer 
        model.add(Flatten())
        model.add(Dense(200,activation='relu',name='fc1'))
        # Dropout on the fully connected layer (1 in 2 probability of dropout) 
        model.add(Dropout(0.5,name='drop2'))
        # Single unit output layer with sigmoid nonlinearity 
        model.add(Dense(1,activation='sigmoid',name='fc2'))
        # Use Stochastic Gradient Discent for optimization 
        sgd = SGD(learning_rate=0.1,decay=0.005,nesterov=False)
        model.compile(optimizer=sgd,loss='binary_crossentropy',metrics=['accuracy'])
        return model

    def fit(self, X, y, batch_size=128, epochs=10):
        self.model = self.create_model()
        self.model.fit(X, y, batch_size=batch_size, epochs=epochs)

    def predict_proba(self, X):
        y_pred = self.model.predict(X)
        return np.column_stack((1 - y_pred, y_pred))

    def predict(self, X):
        return np.round(self.model.predict(X))

    def get_params(self, deep=True):
        return {'learning_rate': self.learning_rate, 'activation': self.activation}

    def set_params(self, **params):
        self.learning_rate = params['learning_rate']
        self.activation = params['activation']
        self.optimizer = params['optimizer']
        return self

In [ ]:
# Define the hyperparameters to search
param_dist = {'learning_rate': [0.1, 0.01],
            'batch_size': [128, 2*128, 3*128],
            'epochs': [20, 30, 40],
            'optimizer': [SGD, Adam, Adagrad, RMSprop],
            'activation': ['tanh', 'relu', 'sigmoid']}

# Create the scoring function using the f1_score
f1_scorer = make_scorer(f1_score, average='micro')


# Create the GridSearch object
grid_search = GridSearchCV(CNNModel(), param_grid=param_dist, scoring=f1_scorer, cv=2, verbose=3, return_train_score=True)
# Fit the randomized search object to the training data
print('Fitting model...')
grid_result = grid_search.fit(X_train, Y_train)

print('Best: %f using %s' % (grid_result.best_score_,grid_result.best_params_))
# Print mean and standard deviation of accuracy scores for each combination
# of parameters evaluated by GridSearchCV
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean,std,param in zip(means,stds,params):
    print('%f (%f) with: %r' % (mean,std,param))

# save the best model
print('Saving the best model...')
best_model = grid_result.best_estimator_
dump(best_model, 'best_model.joblib')

In [ ]:
print("Generating predictions...")
Y_predV = best_model.predict(X_testV)
Y_predH = best_model.predict(X_testH)
Y_pred = Y_predV + Y_predH
Y_pred[Y_pred>1] = 1

# reshape Y_pred to be a list of integers
Y_pred = Y_pred.reshape(Y_pred.shape[0])
Y_pred = [int(round(y)) for y in Y_pred]


In [ ]:
# create submission file
print("Creating submission file...")
submission = pd.DataFrame({'id':file_id,'label':Y_pred})
submission.to_csv('submission.csv',index=False)
print("Done")